In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import from_unixtime, when, col, lit

In [2]:
import json

In [3]:
sc = SparkContext('local')
spark = SparkSession(sc)

In [4]:
#%reset
import pyspark.sql
session = pyspark.sql.SparkSession.builder \
    .master('spark://10.64.22.198:7077') \
    .appName('Demo') \
    .config('spark.jars.packages','org.diana-hep:spark-root_2.11:0.1.16,org.diana-hep:histogrammar-sparksql_2.11:1.0.4') \
    .config('spark.driver.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('spark.executor.extraClassPath','/opt/hadoop/share/hadoop/common/lib/EOSfs.jar') \
    .config('py-files','helper.py') \
    .getOrCreate()
    
sqlContext = session

print('SparkSQL sesssion created')

SparkSQL sesssion created


In [5]:
from pyspark.sql.functions import lit
from samples_tthml import *


In [6]:
for s in samples:
    print(BASE,samples[s]['filename'])

../../Files/ttH_Multilep/ mc16_13TeV.aMcAtNloPythia8EvtGen_ttH_r9364_p3832.csv
../../Files/ttH_Multilep/ mc16_13TeV.Sherpa_221_NN30NNLO_ttW_multilegNLO_r9364_p3830.csv


In [7]:
DFList = [] 

label_count=0
for s in samples:
    dsPath = BASE+samples[s]['filename']
    print('Loading {0} sample file'.format(s),'\n',dsPath )   
    tempDF =sqlContext.read\
                .format('csv')\
                .option('header', 'true')\
                .option('inferschema', 'true')\
                .load(dsPath)
    tempDF=tempDF.withColumn('sample', lit(s)) 
    tempDF=tempDF.withColumn('label', lit(label_count)) 
    label_count+=1
    DFList.append(tempDF)

Loading ttH sample file 
 ../../Files/ttH_Multilep/mc16_13TeV.aMcAtNloPythia8EvtGen_ttH_r9364_p3832.csv
Loading ttW sample file 
 ../../Files/ttH_Multilep/mc16_13TeV.Sherpa_221_NN30NNLO_ttW_multilegNLO_r9364_p3830.csv


In [8]:
DFList[0].printSchema()

root
 |-- entry: integer (nullable = true)
 |-- DEtall01: double (nullable = true)
 |-- lep_flavour: integer (nullable = true)
 |-- max_eta: double (nullable = true)
 |-- minDeltaR_LJ_0: double (nullable = true)
 |-- minDeltaR_LJ_1: double (nullable = true)
 |-- Meff: double (nullable = true)
 |-- DRlj00: double (nullable = true)
 |-- min_DRl0b: double (nullable = true)
 |-- min_DRlj_new: double (nullable = true)
 |-- DPhij0MET: double (nullable = true)
 |-- lead_BjetPt: double (nullable = true)
 |-- lead_jet_sumBEff: double (nullable = true)
 |-- sublead_jet_sumBEff: double (nullable = true)
 |-- scale_nom: double (nullable = true)
 |-- pileupEventWeight_090: double (nullable = true)
 |-- MV2c10_70_EventWeight: double (nullable = true)
 |-- JVT_EventWeight: double (nullable = true)
 |-- SherpaNJetWeight: double (nullable = true)
 |-- EventNumber: integer (nullable = true)
 |-- RunYear: integer (nullable = true)
 |-- MET_RefFinal_et: double (nullable = true)
 |-- lep_Pt_0: double (null

In [9]:
total_events = DFList[0].count()
print('There are',total_events,'events')

There are 69902 events


In [10]:
with open("../VarList_30.json") as vardict:
    variablelist = json.load(vardict)[:]
    
variablelist.append("sample")
variablelist.append("label")
variablelist

['DEtall01',
 'lep_flavour',
 'max_eta',
 'minDeltaR_LJ_0',
 'minDeltaR_LJ_1',
 'Meff',
 'DRlj00',
 'min_DRl0b',
 'min_DRlj_new',
 'DPhij0MET',
 'lead_BjetPt',
 'lead_jet_sumBEff',
 'sublead_jet_sumBEff',
 'MET_RefFinal_et',
 'lep_Pt_0',
 'lep_Eta_0',
 'lep_Phi_0',
 'lep_Pt_1',
 'lep_Eta_1',
 'lep_Phi_1',
 'total_charge',
 'Mll01',
 'Ptll01',
 'DRll01',
 'nJets_OR_T',
 'nJets_OR_T_MV2c10_70',
 'HT_lep',
 'HT_jets',
 'lead_jetPt',
 'sublead_jetPt',
 'sample',
 'label']

In [11]:
# Select columns from dataframe
DF = DFList[0].select(variablelist)
DF.printSchema()

root
 |-- DEtall01: double (nullable = true)
 |-- lep_flavour: integer (nullable = true)
 |-- max_eta: double (nullable = true)
 |-- minDeltaR_LJ_0: double (nullable = true)
 |-- minDeltaR_LJ_1: double (nullable = true)
 |-- Meff: double (nullable = true)
 |-- DRlj00: double (nullable = true)
 |-- min_DRl0b: double (nullable = true)
 |-- min_DRlj_new: double (nullable = true)
 |-- DPhij0MET: double (nullable = true)
 |-- lead_BjetPt: double (nullable = true)
 |-- lead_jet_sumBEff: double (nullable = true)
 |-- sublead_jet_sumBEff: double (nullable = true)
 |-- MET_RefFinal_et: double (nullable = true)
 |-- lep_Pt_0: double (nullable = true)
 |-- lep_Eta_0: double (nullable = true)
 |-- lep_Phi_0: double (nullable = true)
 |-- lep_Pt_1: double (nullable = true)
 |-- lep_Eta_1: double (nullable = true)
 |-- lep_Phi_1: double (nullable = true)
 |-- total_charge: integer (nullable = true)
 |-- Mll01: double (nullable = true)
 |-- Ptll01: double (nullable = true)
 |-- DRll01: double (nullab

In [12]:
# Merge all dataset into a single dataframe
for df_ in DFList[1:]:
    DF = DF.union(df_.select(variablelist))

print( 'Partitions: {}'.format(DF.rdd.getNumPartitions()))

Partitions: 2


In [13]:
print('total number of events in the DataFrame  = ', DF.count())
print( 'events in the DataFrame with \"nJets_OR_T > 5\" = ', DF.filter('nJets_OR_T > 5').count())

total number of events in the DataFrame  =  156656
events in the DataFrame with "nJets_OR_T > 5" =  38368


cach dataframe into memory, shared across the Spark cluster nodes

In [14]:
DF = DF.cache()

In [15]:
DF.filter(DF['sample'] == 'ttH')\
  .select('sample','nJets_OR_T','lep_Pt_0','lep_Eta_0','lep_Phi_0','total_charge')\
  .show(5)

+------+----------+---------+-----------+-----------+------------+
|sample|nJets_OR_T| lep_Pt_0|  lep_Eta_0|  lep_Phi_0|total_charge|
+------+----------+---------+-----------+-----------+------------+
|   ttH|         6| 65184.87|  1.3295902|   2.048432|          -2|
|   ttH|         5|40491.316|-0.41755068|-0.65665025|           2|
|   ttH|         4| 80014.03|0.043774348| -1.9746306|          -2|
|   ttH|         5|35063.797|  1.3237612|   3.133612|           2|
|   ttH|         7| 45864.24|  1.9134886| -1.6473318|           2|
+------+----------+---------+-----------+-----------+------------+
only showing top 5 rows



In [16]:
DF.groupBy('label').count().show(2)

+-----+-----+
|label|count|
+-----+-----+
|    1|86754|
|    0|69902|
+-----+-----+



In [17]:
balance_of_classes = DF.filter(DF['label'] == 0).count()/DF.filter(DF['label'] == 1).count()

In [18]:
if balance_of_classes > 0.5 or balance_of_classes < 1.5:
    print("rather balanced ds. split to it into train and test dataframes.")
    train, test = DF.randomSplit([0.9,0.1])

rather balanced ds. split to it into train and test dataframes.


In [19]:
print('Events for the training {}'.format(train.count()))
print('Events for the validation {}'.format(test.count()))

Events for the training 140820
Events for the validation 15836


In [20]:
import matplotlib.pyplot as plt 
from pyspark.sql.functions import col
%matplotlib notebook

def compute_hist(data, feature, target='label', n_bins=100, x_lim=[0,3]):
        
    ## Fix the range
    data = data.where((col(feature)<=x_lim[1]) &
                      (col(feature)>=x_lim[0]))
    
    sgn = data.where(col(target)==1.0) 
    bkg = data.where(col(target)==0.0)

    ## Compute the histograms
    bins_sgn, counts_sgn = sgn.select(feature).rdd.flatMap(lambda x: x).histogram(n_bins)
    bins_bkg, counts_bkg = bkg.select(feature).rdd.flatMap(lambda x: x).histogram(n_bins)
    
    return (bins_sgn, counts_sgn), (bins_bkg, counts_bkg)

In [21]:
train.filter('label==0')

DataFrame[DEtall01: double, lep_flavour: int, max_eta: double, minDeltaR_LJ_0: double, minDeltaR_LJ_1: double, Meff: double, DRlj00: double, min_DRl0b: double, min_DRlj_new: double, DPhij0MET: double, lead_BjetPt: double, lead_jet_sumBEff: double, sublead_jet_sumBEff: double, MET_RefFinal_et: double, lep_Pt_0: double, lep_Eta_0: double, lep_Phi_0: double, lep_Pt_1: double, lep_Eta_1: double, lep_Phi_1: double, total_charge: int, Mll01: double, Ptll01: double, DRll01: double, nJets_OR_T: int, nJets_OR_T_MV2c10_70: int, HT_lep: double, HT_jets: double, lead_jetPt: double, sublead_jetPt: double, sample: string, label: int]

In [22]:
hist_signal, hist_bkg = compute_hist(data=train, feature='Mll01', target='label', n_bins=50, x_lim=[0,3])


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/py4j/java_gateway.py", line 1159, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:56488)
Traceback (most recent c

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:56488)
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyspark/rdd.py", line 816, in collect
    sock_info = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyspark/sql/utils.py", line 63, in deco
    return f(*a, **kw)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/py4j/protocol.py", line 336, in get_return_value
    format(target_id, ".", name))
py4j.protocol.Py4JError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe

During handling of the above exception, 

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:56488)

In [ ]:

f, ax = plt.subplots()
ax.hist(hist_signal[0][:-1], bins=hist_signal[0], weights=hist_signal[1], alpha=0.5, label='signal')
ax.hist(hist_bkg[0][:-1], bins=hist_bkg[0], weights=hist_bkg[1], alpha=0.5, label='background')
ax.set_xlabel('$m_{bb}$')
ax.set_ylabel('counts')
ax.set_title("Distribution of $m_{bb}$")
ax.legend()
plt.show()

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=variablelist, outputCol='features')
train = assembler.transform(train)
test = assembler.transform(test)